In [24]:
# 0.A IMPORT ALL USED LIBRARIES
from random import randint
import numpy as np
from cplex import Cplex
import cplex
from itertools import combinations

In [25]:
NBR_CITIES = 5
MIN_DISTANCE_BETWEEN_CITIES = 1
MAX_DISTANCE_BETWEEN_CITIES = 15

def build_random_instance(nbr_cities=6, min_distance=1, max_distance=15):
    start = randint(0, nbr_cities-1) # 1. random generate the starting city
    distances = np.random.uniform(min_distance, max_distance, (nbr_cities, nbr_cities)).astype(int) # 2. random generate a matrix of (integer) paths between cities
    distances = np.triu(distances, k=1)
    distances += distances.T # 3. the distance between city a -> city b should be equal to the between city b -> city a
    np.fill_diagonal(distances, 0) # 4. replace all distance for a city to itself (the diagonal) to 0 
    return distances, start

distances, start = build_random_instance(NBR_CITIES, MIN_DISTANCE_BETWEEN_CITIES, MAX_DISTANCE_BETWEEN_CITIES)
print(distances)
print(start)

[[ 0 11 13 11  7]
 [11  0 12  1  7]
 [13 12  0  8  3]
 [11  1  8  0  3]
 [ 7  7  3  3  0]]
1


In [26]:
COMPUTING_TIME_LIMIT = 300 #seconds
MEMORY_LIMIT = 1024 #Gb
NODE_LIMIT = 1000 #size of the branch&bound/cut decision tree
MIP_TOLERANCE = 0 #%
MAX_NBR_THREADS = 4

def build_model(distances, nbr_cities=6):
    m = Cplex()

    # 1. Create an array of variables with names and lower/upper bounds
    names = [f"path_{i}_{j}" for i in range(nbr_cities) for j in range(nbr_cities) if j != i]
    print(names)
    types = [m.variables.type.binary] * len(names)
    lb = [0] * len(names)
    up = [1] * len(names)
    m.variables.add(names=names, types=types, lb=lb, ub=up) 

    # 2. Create the min objective function
    for i in range(nbr_cities-1):
        for j in range(i+1, nbr_cities):
            m.objective.set_linear(f"path_{i}_{j}", int(distances[i][j]))
            m.objective.set_linear(f"path_{j}_{i}", int(distances[j][i]))
    m.objective.set_sense(m.objective.sense.minimize)

    # 3. Create the first two constraint: exactly on selected path FROM and TO each city
    for i in range(nbr_cities):
        paths_from_i = [f"path_{i}_{j}" for j in range(nbr_cities) if j != i]
        paths_to_i = [f"path_{j}_{i}" for j in range(nbr_cities) if j != i]
        m.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=paths_from_i, val=[1]*len(paths_from_i))], senses=['E'], rhs=[1])
        m.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=paths_to_i, val=[1]*len(paths_to_i))], senses=['E'], rhs=[1])

    # 4. Apply the CPLEX runtime configuration
    m.parameters.timelimit.set(COMPUTING_TIME_LIMIT)
    m.parameters.workmem.set(MEMORY_LIMIT)
    m.parameters.mip.limits.nodes.set(NODE_LIMIT)
    m.parameters.mip.tolerances.mipgap.set(MIP_TOLERANCE)
    m.parameters.threads.set(MAX_NBR_THREADS)
    return m

model = build_model(distances, NBR_CITIES)

['path_0_1', 'path_0_2', 'path_0_3', 'path_0_4', 'path_1_0', 'path_1_2', 'path_1_3', 'path_1_4', 'path_2_0', 'path_2_1', 'path_2_3', 'path_2_4', 'path_3_0', 'path_3_1', 'path_3_2', 'path_3_4', 'path_4_0', 'path_4_1', 'path_4_2', 'path_4_3']


In [27]:
def generate_subsets(nbr_cities=6):
    subsets = []
    for size in range(2,nbr_cities):
        subsets.extend(combinations(range(nbr_cities), size))
    return subsets

def add_last_constraint(m, nbr_cities=6):
    subsets = generate_subsets(nbr_cities)
    for s in subsets:
        paths = [f"path_{i}_{j}" for i in s for j in s if j != i]
        print(paths)
        print(len(s)-1)
        m.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=paths, val=[1]*len(paths))], senses=['L'], rhs=[len(s)-1])
    return m

model = add_last_constraint(model, NBR_CITIES)

['path_0_1', 'path_1_0']
1
['path_0_2', 'path_2_0']
1
['path_0_3', 'path_3_0']
1
['path_0_4', 'path_4_0']
1
['path_1_2', 'path_2_1']
1
['path_1_3', 'path_3_1']
1
['path_1_4', 'path_4_1']
1
['path_2_3', 'path_3_2']
1
['path_2_4', 'path_4_2']
1
['path_3_4', 'path_4_3']
1
['path_0_1', 'path_0_2', 'path_1_0', 'path_1_2', 'path_2_0', 'path_2_1']
2
['path_0_1', 'path_0_3', 'path_1_0', 'path_1_3', 'path_3_0', 'path_3_1']
2
['path_0_1', 'path_0_4', 'path_1_0', 'path_1_4', 'path_4_0', 'path_4_1']
2
['path_0_2', 'path_0_3', 'path_2_0', 'path_2_3', 'path_3_0', 'path_3_2']
2
['path_0_2', 'path_0_4', 'path_2_0', 'path_2_4', 'path_4_0', 'path_4_2']
2
['path_0_3', 'path_0_4', 'path_3_0', 'path_3_4', 'path_4_0', 'path_4_3']
2
['path_1_2', 'path_1_3', 'path_2_1', 'path_2_3', 'path_3_1', 'path_3_2']
2
['path_1_2', 'path_1_4', 'path_2_1', 'path_2_4', 'path_4_1', 'path_4_2']
2
['path_1_3', 'path_1_4', 'path_3_1', 'path_3_4', 'path_4_1', 'path_4_3']
2
['path_2_3', 'path_2_4', 'path_3_2', 'path_3_4', 'path_

In [28]:
print("\nConstraints:")
for i in range(model.linear_constraints.get_num()):
    row = model.linear_constraints.get_rows(i)
    senses = "<=" if model.linear_constraints.get_senses(i) == 'L' else ">=" if model.linear_constraints.get_senses(i) == 'G' else "="
    rhs = model.linear_constraints.get_rhs(i)
    constraint_expr = " + ".join(f"{model.variables.get_names(var)}" for var, _ in zip(row.ind, row.val))
    print(f"{constraint_expr} {senses} {rhs}")

print("\nObjective:")
objective = model.objective.get_linear()
obj_expr = " + ".join(f"{coef}*{var_name}" for coef, var_name in zip(objective, model.variables.get_names()))
print(f"Minimize {obj_expr}")


Constraints:
path_0_1 + path_0_2 + path_0_3 + path_0_4 = 1.0
path_1_0 + path_2_0 + path_3_0 + path_4_0 = 1.0
path_1_0 + path_1_2 + path_1_3 + path_1_4 = 1.0
path_0_1 + path_2_1 + path_3_1 + path_4_1 = 1.0
path_2_0 + path_2_1 + path_2_3 + path_2_4 = 1.0
path_0_2 + path_1_2 + path_3_2 + path_4_2 = 1.0
path_3_0 + path_3_1 + path_3_2 + path_3_4 = 1.0
path_0_3 + path_1_3 + path_2_3 + path_4_3 = 1.0
path_4_0 + path_4_1 + path_4_2 + path_4_3 = 1.0
path_0_4 + path_1_4 + path_2_4 + path_3_4 = 1.0
path_0_1 + path_1_0 <= 1.0
path_0_2 + path_2_0 <= 1.0
path_0_3 + path_3_0 <= 1.0
path_0_4 + path_4_0 <= 1.0
path_1_2 + path_2_1 <= 1.0
path_1_3 + path_3_1 <= 1.0
path_1_4 + path_4_1 <= 1.0
path_2_3 + path_3_2 <= 1.0
path_2_4 + path_4_2 <= 1.0
path_3_4 + path_4_3 <= 1.0
path_0_1 + path_0_2 + path_1_0 + path_1_2 + path_2_0 + path_2_1 <= 2.0
path_0_1 + path_0_3 + path_1_0 + path_1_3 + path_3_0 + path_3_1 <= 2.0
path_0_1 + path_0_4 + path_1_0 + path_1_4 + path_4_0 + path_4_1 <= 2.0
path_0_2 + path_0_3 + p

In [30]:
# V. RUN THE MODEL AND DISPLAY THE FINAL SOLUTION
model.solve()

# 1. Display the value of the objective function
print("OBJECTIVE FUNCTION VALUE: ", model.solution.get_objective_value())

# 2. Dumb display of final value of each decision variable
def dumb_display(model):
    for name, value in zip(model.variables.get_names(), model.solution.get_values()):
        if(value>=1):
            print(f"{name}: {value}")

dumb_display(model)

# 3. Smart display the complete seletected path
def search_next_city(var_names, var_values, current_city=0, nbr_cities=6):
    paths_from_current_city = [f"path_{current_city}_{i}" for i in range(nbr_cities) if i != current_city]
    for name, value in zip(var_names, var_values):
        if name in paths_from_current_city and value >= 1:
            return int(name.split("_")[2])
    return -1

def display_path(model, distances, start=0, nbr_cities=6):
    first_itr = True
    current_city = start
    next_city = -1
    print("COMPLETE PATH:", end=' ')
    while first_itr or (next_city != -1 and current_city != start):
        next_city = search_next_city(model.variables.get_names(), model.solution.get_values(), current_city, nbr_cities)
        if(next_city != -1):
            print("City_"+str(current_city)+" -> ("+str(distances[current_city][next_city])+") ->", end=' ')
            current_city = next_city
        else:
            print("City_"+str(current_city), end=' ')
        first_itr = False
    print("City_"+str(start))

display_path(model, distances, start, NBR_CITIES)

Version identifier: 22.1.0.0 | 2022-03-27 | 54982fbec
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 4
CPXPARAM_MIP_Limits_Nodes                        1000
CPXPARAM_TimeLimit                               300
CPXPARAM_WorkMem                                 1024
CPXPARAM_MIP_Tolerances_MIPGap                   0

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.00 ticks)
Parallel b&c, 4 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.00 ticks)
OBJECTIVE FUNCTION VALUE:  30.0
path_0_4: 1.0
path_1_0: 1.0
path_2_3: 1.0
path_3_1: 1.0
path_4_2: 1.0
COMPLETE PATH: City_1 -> (11) -> City_0 -> (7) -> City_4 -> (3) -> City_2 -> (8) -> City_3 -> (1) -> City_1
